In [1]:
import math
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
from __future__ import division
mpl.rc('figure', figsize=[10,6])

In [2]:
def updateFlag(val,sorted_attr_row):
    for i in range(len(sorted_attr_row)-1):
        val[i] = sorted_attr_row[i+1]

def defineT(attr):
    dtype = [('Id',int)]
    for i in range(len(attr)):
        dtype.append((attr[i],'S10'))
    return dtype

# Grouping by node attributes
def ACompatible(graph,edge,attr):
    dtype = defineT(attr)
    value = list(graph[attr].itertuples())
    original_table = np.asarray(value,dtype=dtype)
    groupArr = []

    sorted_attr = np.sort(original_table, order=attr)  
    val = ['','']
    for i in range(len(sorted_attr)):
        for j in range(len(attr)):
            if(sorted_attr[i][j+1]==val[j]):
                if(j == len(attr)-1):
                    newGroup.append(sorted_attr[i])
            else:
                newGroup = [(sorted_attr[i])]
                groupArr.append(newGroup)
                updateFlag(val,sorted_attr[i])
                break
    return groupArr

In [3]:
# Update data structures
def DataStructure(tempResult, tempVertex, edge):
    groupNum = len(tempResult)
    nodeNum = edge.shape[0]
    
    # in-groups bitMap
    mapsize = (nodeNum,groupNum)
    bitMap = np.zeros(mapsize)
    
    for i in range(nodeNum):
        for g in range(groupNum):
            groupSet = tempResult[g]
            for j in range(len(groupSet)):
                currentNode = groupSet[j][0]
                # update group index of each node
                tempVertex[currentNode][2] = g
                # update number of nodes from in-groups
                if(not pd.isnull(edge[currentNode][i+1])):
                    bitMap[i][g] = bitMap[i][g] + 1
    return tempVertex, bitMap

In [4]:
# Split into subgroups
def Split(node, tempResult, tempVertex, tempBitMap):
    groupNum = tempBitMap.shape[1]
    currentGroup = tempVertex[node][2]
    groupSet = tempResult[currentGroup]
    subGroup1 = []
    subGroup2 = []
    
    for i in range(len(groupSet)):
        currentNode = groupSet[i][0]
        for g in range(groupNum):
            if(tempBitMap[currentNode][g] != tempBitMap[node][g]):
                # Split to a new group
                tempVertex[currentNode][2] = groupNum
        if(tempVertex[currentNode][2] == currentGroup):
            subGroup1.append((currentNode, tempVertex[currentNode][1]))
        elif(tempVertex[currentNode][2] == groupNum):
            subGroup2.append((currentNode, tempVertex[currentNode][1]))
        
    if(subGroup2):
        tempResult.remove(groupSet)
        tempResult.append(subGroup1)
        tempResult.append(subGroup2)
        return True, tempVertex, tempResult
    else:
        return False, tempVertex, tempResult

In [5]:
# BFS
def BFS(edge, source):
    visited, queue = [source], [source]
    while queue:
        node = queue.pop(0)
        for i in range(1, edge.shape[1]):
            if not pd.isnull(edge[node][i]):
                neighbor = i - 1
                if neighbor not in visited:
                    visited.append(neighbor)
                    queue.append(neighbor)
    return visited

In [6]:
# Graph summarization algorithm
def ProvSummary(src, vertex, edge, attr):
    ACompResult = ACompatible(vertex, edge, attr)
    print "A-Compatible result: ", ACompResult
    vertex = np.asarray(vertex)
    edge = np.asarray(edge)
    queue = BFS(edge, src)
    print "queue: ", queue
    updateVertex, updateBitMap = DataStructure(ACompResult, vertex, edge)
    splitVertex = updateVertex
    splitResult = ACompResult
    while queue:
        node = queue.pop(0)
        # split node
        isSplit, splitVertex, splitResult = Split(node, ACompResult, updateVertex, updateBitMap)
        if isSplit:
            # update data structures
            updateVertex, updateBitMap = DataStructure(splitResult, splitVertex, edge)
    return splitResult

In [8]:
# Test on an example
vertex = pd.read_csv('testData/vertices.csv')
edge = pd.read_csv('testData/edges.csv')
attr = ['Type']
source = 5
ProvSummary(source, vertex, edge, attr)

A-Compatible result:  [[(1, 'A'), (3, 'A'), (5, 'A')], [(0, 'E'), (2, 'E'), (4, 'E')]]
queue:  [5, 0, 4, 1, 3, 2]


[[(5, 'A')], [(1, 'A'), (3, 'A')], [(0, 'E'), (4, 'E')], [(2, 'E')]]

# swift data

In [9]:
# Normalize vertex and edge
def Normalize(vertex, edge):
    NodeNum = len(vertex)
    EdgeNum = len(edge)
    vertex = np.asarray(vertex)
    edge = np.asarray(edge)
    
    # create the adjencent matrix for edge
    size = (NodeNum,NodeNum+1)
    AdjMatrix = np.empty(size, dtype=object)
    for k in range(NodeNum):
        AdjMatrix[k][0] = k+1
    for i in range(EdgeNum):
        if edge[i][2]==1:
            for j in range(NodeNum):
                if (vertex[j][0] == edge[i][0]): 
                    startV = j
                if(vertex[j][9] == edge[i][1]):
                    endV = j
        else:
            for j in range(NodeNum):
                if (vertex[j][9] == edge[i][1]):      
                    startV = j
                if(vertex[j][0] == edge[i][0]):
                    endV = j
        AdjMatrix[startV][endV+1] = 1
    edge = AdjMatrix
    
    size = (NodeNum, 3)
    reVertex = np.empty(size, dtype=object)
    for i in range(NodeNum):
        reVertex[i][0] = i
        reVertex[i][1] = vertex[i][14]
        
    return reVertex, edge

In [10]:
# Algorithm for swift data
def ProvSummary_Swift(src, vertex, edge, attr):
    ACompResult = ACompatible(vertex, edge, attr)
    print "A-Compatible result: ", ACompResult
    vertex, edge = Normalize(vertex, edge)
    queue = BFS(edge, src)
    print "queue: ", queue
    updateVertex, updateBitMap = DataStructure(ACompResult, vertex, edge)
    splitResult = ACompResult
    while queue:
        node = queue.pop(0)
        # split node
        isSplit, splitVertex, splitResult = Split(node, ACompResult, updateVertex, updateBitMap)
        if isSplit:
            # update data structures
            updateVertex, updateBitMap = DataStructure(splitResult, splitVertex, edge)
    print len(splitResult), "groups"
    return splitResult

In [11]:
# Test on swift data
vertex = pd.read_csv('swiftData/vertex.csv')
edge = pd.read_csv('swiftData/edge.csv')
attr = ['type']
source = 18
ProvSummary_Swift(source, vertex, edge, attr)

A-Compatible result:  [[(0, 'App'), (1, 'App'), (2, 'App'), (3, 'App'), (4, 'App'), (5, 'App'), (6, 'App'), (7, 'App'), (8, 'App'), (9, 'App'), (10, 'App'), (11, 'App'), (12, 'App'), (13, 'App'), (14, 'App'), (15, 'App'), (16, 'App'), (17, 'App')], [(18, 'File'), (19, 'File'), (20, 'File'), (21, 'File'), (22, 'File'), (23, 'File'), (24, 'File'), (25, 'File'), (26, 'File'), (27, 'File'), (28, 'File'), (29, 'File'), (30, 'File'), (31, 'File'), (32, 'File'), (33, 'File'), (34, 'File'), (35, 'File'), (36, 'File'), (37, 'File'), (38, 'File'), (39, 'File'), (40, 'File'), (41, 'File'), (42, 'File'), (43, 'File'), (44, 'File'), (45, 'File'), (46, 'File'), (47, 'File'), (48, 'File'), (49, 'File'), (50, 'File'), (51, 'File'), (52, 'File'), (53, 'File'), (54, 'File'), (55, 'File'), (56, 'File'), (57, 'File'), (58, 'File'), (59, 'File'), (60, 'File')]]
queue:  [18, 0, 19, 20, 21, 22, 23, 24, 25, 26, 1, 9, 3, 12, 2, 10, 7, 16, 6, 14, 8, 15, 5, 13, 4, 11, 27, 28, 29, 52, 33, 34, 35, 55, 30, 31, 32, 

[[(18, 'File'), (51, 'File')],
 [(0, 'App')],
 [(19, 'File'),
  (20, 'File'),
  (21, 'File'),
  (22, 'File'),
  (23, 'File'),
  (24, 'File'),
  (25, 'File'),
  (26, 'File')],
 [(1, 'App'),
  (2, 'App'),
  (3, 'App'),
  (4, 'App'),
  (5, 'App'),
  (6, 'App'),
  (7, 'App'),
  (8, 'App')],
 [(9, 'App'),
  (10, 'App'),
  (11, 'App'),
  (12, 'App'),
  (13, 'App'),
  (14, 'App'),
  (15, 'App'),
  (16, 'App')],
 [(17, 'App')],
 [(27, 'File'),
  (28, 'File'),
  (29, 'File'),
  (30, 'File'),
  (31, 'File'),
  (32, 'File'),
  (33, 'File'),
  (34, 'File'),
  (35, 'File'),
  (36, 'File'),
  (37, 'File'),
  (38, 'File'),
  (39, 'File'),
  (40, 'File'),
  (41, 'File'),
  (42, 'File'),
  (43, 'File'),
  (44, 'File'),
  (45, 'File'),
  (46, 'File'),
  (47, 'File'),
  (48, 'File'),
  (49, 'File'),
  (50, 'File')],
 [(52, 'File'),
  (53, 'File'),
  (54, 'File'),
  (55, 'File'),
  (56, 'File'),
  (57, 'File'),
  (58, 'File'),
  (59, 'File')],
 [(60, 'File')]]